# Describe a PIV recording


Let's say you recorded multiple PIV images and put everything into a *ZIP* folder. The following shows how such a dataset can be described. As a real-world example we use a dataset from the "PIV Challenge":

In [1]:
from pivmetalib.prov import Person
from pivmetalib.dcat import Dataset, Distribution
from pivmetalib import PivImageDistribution
from pivmetalib.namespace import PIVMETA 

In [2]:
ds = Dataset(
    title='piv-challenge-1-C',
    creator=Person(last_name='Okamoto', mbox="okamoto@tokai.t.u-tokyo.ac.jp"),
    modified="2000-10-28",
    landingPage="https://www.pivchallenge.org/pub/index.html#c",
    description="Different velocity gradients with spatially varying image quality (provided Okamoto) < synthetic > [256 x 128]",
    distribution=[
        PivImageDistribution(title='raw data',
                        download_URL='https://www.pivchallenge.org/pub/C/C.zip',
                        media_type='application/zip',
                        piv_image_type=PIVMETA.SyntheticImage),
        Distribution(title='ReadMe file',
                     download_URL='https://www.pivchallenge.org/pub/E/readmeE.txt'),
    ]
)

In [3]:
print(ds.dump_jsonld(id='https://www.pivchallenge.org/pub/index.html#c'))

URLError: <urlopen error [Errno 11001] getaddrinfo failed>